261


## 必要なライブラリのimport

In [1]:
import numpy as np
import pandas as pd
import lightgbm as lgb
from sklearn.metrics import mean_squared_error
import pickle

pd.set_option('display.max_columns', 500)

In [2]:
# 予測したいデータの読み込み

path_test = '../../../../data/extornal/test_processed_clv&churn&ltv.csv'

test_all = pd.read_csv(path_test)

# 特徴量の選択
features_all = pd.read_csv('../../../../data/processed/features_61_all.csv')


In [3]:
test = test_all

n_folds = 5
predictions = np.zeros(len(test))
features = [c for c in test.columns if c not in ['card_id',]]
#categorical_feats = ['feature_1','feature_2', 'feature_3','hist_merchant_category_nu','hist_merchant_category_mode','hist_city_nu','hist_city_mode','hist_state_nu','hist_state_mode','hist_category_1_mod','hist_category_2_mod','hist_category_3_mod','new_merchant_category_nu','new_merchant_category_mode','new_city_nu','new_city_mode','new_state_nu','new_state_mode','new_category_1_mod','new_category_2_mod','new_category_3_mod']


for fold_ in range(n_folds):
    # モデルを読み込む
    with open(f'../src/models/lgb_61/model_fold_{fold_}.pkl', 'rb') as f:
        clf = pickle.load(f)
    # モデルで予測を行う
    fold_predictions = clf.predict(test[features])
    # 予測結果を足し合わせる
    predictions += fold_predictions

# 平均を取る
predictions /= n_folds

# RMSEを計算する（テストデータに対して真の値がある場合）
# y_testが存在する場合に限り
if 'y_test' in locals():
    score = mean_squared_error(y_test, predictions, squared=False)
    print(f'RMSE={score}')

In [4]:
# 予測結果を出力する
print(predictions)

[-3.62296341 -0.33087896 -1.2837031  ...  0.88702273 -4.18881929
  0.13700029]


In [5]:
sub_df = pd.DataFrame({"card_id":test["card_id"].values})
sub_df["target"] = predictions
sub_df.to_csv("../submit_oshikiri_61.csv", index=False)

In [9]:
# 特徴量の選択
pct25 = features_all['importance'].quantile(0.25)


In [10]:
features_selected = features_all.query('importance > @pct25').reset_index(drop=True)
selected_columns = features_selected['feature'].tolist()
selected_columns.insert(0,'card_id')
test = test_all[selected_columns]

In [13]:
test

,card_id,category_1_installments_mean,hist_day_min,new_purchase_amount_mean,category_1_installments_min,total_state_nu,elapsed_time,auth_category_2_5.0_mean,hist_amount_mean,new_amount_mean,auth_state_id_nunique,hist_amount_month_ratio_mean,hist_amount_max,hist_ratio_Yes,installments_count_std,hist_duration_mean,total_price_max,total_day_max,new_purchase_amount_min,new_category_2_1.0_mean,new_merchant_category_id_nunique,total_amount_mean,feature_var,history_city_id_nunique,new_installments_mean_y,installments_purchase_amount_mean,feature_sum,category_1_purchase_amount_mean,history_category_3_1.0_mean,hist_duration_max,hist_day_max,auth_purchase_month_min,auth_category_2_4.0_mean,total_duration_mean,total_amount_month_ratio_mean,history_purchase_month_min,quarter,installments_count_mean,hist_amount_month_ratio_max,auth_purchase_month_max,total_state_mode,total_amount_min,hist_city_nu,total_amount_total,hist_day_nu,history_month_lag_max,auth_city_id_nunique,total_city_nu,total_day_nu,new_state_mode,hist_month_lag_median,history_purchase_month_max,new_weekend_mean,installments_min_mean,category_1_purchase_amount_max,hist_price_sum,history_state_id_nunique,history_category_3_3.0_mean,auth_purchase_amount_max,history_subsector_id_nunique,auth_installments_mean,new_amount_min,total_price_min,hist_count_No,auth_installments_std,hist_state_mode,hist_frequency,feature_1,city_id_purchase_amount_mean,new_installments_std,installments_mean_mean,installments_max_std,installments_max_mean,total_month_lag_median,installments_purchase_amount_std,history_merchant_category_id_nunique,new_price_mean,installments_purchase_amount_max,history_merchant_id_nunique,new_month_lag_std,category_1_purchase_amount_std,history_month_lag_min,hist_amount_total,total_amount_month_ratio_min,hist_installments_mean,new_duration,total_frequency,auth_subsector_id_nunique,new_purchase_month_min,new_amount_median,category_1_installments_std,auth_purchase_amount_min,hist_LTV,auth_merchant_category_id_nunique,total_installments_mean,auth_transactions_count,new_price_max,installments_min_std,new_amount_month_ratio_min,installments_mean_std,history_installments_mean,total_merchant_category_nu,new_category_1_sum,new_month_diff_mean,new_amount_month_ratio_mean,installments_std_mean,new_month_lag_mean_y,hist_merchant_category_nu,auth_purchase_amount_std,days_feature3_ratio,auth_category_3_3.0_mean,installments_sum_std,hist_monetary_value,installments_sum_mean,city_id_purchase_amount_min,auth_installments_sum,total_recency_rfm,new_price_min,total_city_mode,new_amount_total,new_purchase_month_max,category_1_purchase_amount_min,hist_city_mode,history_category_2_1.0_mean,total_LTV,auth_purchase_amount_mean,ratio_amount_mean,new_clv,new_category_3_2.0_mean,hist_date_min,installments_purchase_amount_min,total_installments_sum,hist_price_mean,first_active_month,city_id_purchase_amount_max,total_monetary_value,hist_price_min,history_category_3_2.0_mean,hist_frequency_rmf,new_merchant_category_mode,hist_merchant_category_mode,hist_price_max,total_duration,new_price_sum,auth_purchase_amount_sum,new_duration_mean,new_city_mode,total_predicted_monetary_value,hist_amount_month_ratio_min,hist_duration_min,new_amount_max,city_id_purchase_amount_std,total_frequency_rfm,new_weekday_mean,history_purchase_amount_max,auth_category_3_2.0_mean,hist_weekend_mean,purchase_amount_max_std,total_duration_min,purchase_amount_mean_mean,history_category_1_mean,total_merchant_category_mode,total_price_sum,installments_std_std,purchase_amount_count_std,hist_recency_rmf,new_purchase_month_std,total_weekend_mean,purchase_amount_mean_std,auth_category_2_1.0_mean,hist_installments_sum,history_installments_std,days_feature3,new_predicted_monetary_value,new_purchase_amount_max,new_date_min,new_duration_min,history_purchase_amount_mean,purchase_amount_std_mean,purchase_amount_min_mean,history_purchase_month_mean,purchase_amount_sum_std,hist_weekday_mean,purchase_amount_count_mean,auth_merchant_id_nunique,total_

In [15]:
for i in [25]:
    pct = features_all['importance'].quantile(i/100)
    features_selected = features_all.query('importance > @pct').reset_index(drop=True)
    selected_columns = features_selected['feature'].tolist()
    selected_columns.insert(0,'card_id')
    test = test_all[selected_columns]

    n_folds = 5
    predictions = np.zeros(len(test))
    features = [c for c in test.columns if c not in ['card_id',]]

    print(len(features))

    for fold_ in range(n_folds):
        # モデルを読み込む
        with open(f'../src/models/lgb_61_{i}/model_fold_{fold_}.pkl', 'rb') as f:
            clf = pickle.load(f)
        # モデルで予測を行う
        fold_predictions = clf.predict(test[features])
        # 予測結果を足し合わせる
        predictions += fold_predictions

    # 平均を取る
    predictions /= n_folds
    # 予測結果を出力する
    print(predictions)

    sub_df = pd.DataFrame({"card_id":test["card_id"].values})
    sub_df["target"] = predictions
    sub_df.to_csv(f"../submit_oshikiri_61_{i}.csv", index=False)

250
[-3.82791924 -0.32204496 -1.20802178 ...  0.83766736 -4.31250046
  0.13487303]
